Python for Everyone!<br/>[Oregon Curriculum Network](http://4dsolutions.net/ocn/)

# Pattern Verification with Python

There's a real difference between verifying a suspected identity, in this case involving arctangents and phi ($\phi$), and providing an algebraic proof.

My impression is some of the Ramanujan convergent series have been verified, which is easier to do through brute force computation, than proved algebraically.

Consider this monster for example:

$$\frac{1}{\pi} = \frac{2\sqrt{2}}{9801} \sum^\infty_{k=0} \frac{(4k)!(1103+26390k)}{(k!)^4 396^{4k}}$$

Has someone provided an algebraic proof?

In the absence of proof, there's still [computational verification](Extended%20Precision.ipynb).

### The Pattern

Here's the pattern David Koski was discovering, using a calculator:

```python
    from math import atan as arctan
    arctan( Ø ** 1) -  arctan( Ø **  -1) == arctan(1/2)
    arctan( Ø **-1) -  arctan( Ø **  -3) == arctan(1/3)
    arctan( Ø **-3) -  arctan( Ø **  -5) == arctan(1/7)
    arctan( Ø **-5) -  arctan( Ø **  -7) == arctan(1/18)
    arctan( Ø **-7) -  arctan( Ø **  -9) == arctan(1/47)
    arctan( Ø **-9) -  arctan( Ø ** -11) == arctan(1/123)
```
. . .

Given the limitations of floating point numbers (the Decimal type does not natively do trig), we may enlist the `unittest.TestCase` method `assertAlmostEqual` to verify the pattern so far.

I'm organizing my tests to run inside of the `verify` function.  That's slightly unusual.  [Here's a link](https://docs.python.org/3/library/unittest.html#organizing-test-code) to the documentation.

In [1]:
from math import atan as arctan, sqrt as rt2
from unittest import TestCase, TextTestRunner, TestSuite
Ø = (1 + rt2(5))/2
    
class TestArcTan(TestCase):
    
    def test_claim(self):
        self.assertAlmostEqual(arctan( Ø**1 ) - arctan( Ø**-1), arctan( 1/2  ))
        self.assertAlmostEqual(arctan( Ø**-1) - arctan( Ø**-3), arctan( 1/3  ))
        self.assertAlmostEqual(arctan( Ø**-3) - arctan(Ø**-5 ), arctan( 1/7  ))
        self.assertAlmostEqual(arctan( Ø**-5) - arctan(Ø**-7 ), arctan( 1/18 ))
        self.assertAlmostEqual(arctan( Ø**-7) - arctan(Ø**-9 ), arctan( 1/47 ))
        self.assertAlmostEqual(arctan( Ø**-9) - arctan(Ø**-11), arctan( 1/123)) 
        
def verify():
    """
    how true are these expressions?
    
    using a function to stuff an empty TestSuit, taking from
    an existing class, is not the usual pattern
    """
    def suite():
        suite = TestSuite()
        suite.addTest(TestArcTan('test_claim'))
        return suite
        
    runner = TextTestRunner()
    runner.run(suite())
    
verify()

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


By inspection, the pattern is fairly clear, but for the sequence of fractions 1/2, 1/3, 1/7, 1/18... in the resulting term.  

A useful resource in such cases is the [On-Line Encyclopedia of Integer Sequences](http://oeis.org/), which comes through for us when we search on just those denominators.  We find this is [a "bisection of the Lucas Numbers"](http://oeis.org/A005248).

We learn from this web page that this sequence may be generated as the sum of consecutive odd-position Fibonacci numbers.  

For example:

In [2]:
def Fibo(a=1, b=0):
    # 1, 0, 1, 1, 2, 3, 5, 8, 13, 21...
    while True:
        yield a
        a, b = b, a + b
        
def A005248():
    """
    Skip half the Fibs, sum consecutive terms from 
    the other half:
    
    1 + 1, 1 + 2, 2 + 5, 5 + 13...
    """
    fibo_0 = Fibo()
    fibo_1 = Fibo()

    # advance 2 positions
    next(fibo_1); next(fibo_1)  #1, 0
    
    while True:
        yield next(fibo_0) + next(fibo_1) # 1 + 1, 1 + 2, 2 + 5...
        next(fibo_0) # skip over next Fib
        next(fibo_1) # skip over next Fib

a005248 = A005248()
print([next(a005248) for _ in range(20)])

[2, 3, 7, 18, 47, 123, 322, 843, 2207, 5778, 15127, 39603, 103682, 271443, 710647, 1860498, 4870847, 12752043, 33385282, 87403803]


Now we have the ability to build successive arctan expressions and verify them.  Rather then depend on unittest, lets simply compute the floating point difference between the two sides of the equation.  If it stays very small, we will consider the expression verified.

In [3]:
def build_expr():
    a005248 = A005248()
    exp1 = 1
    exp2 = -1
    template = "(arctan( Ø ** {}) -  arctan( Ø ** {})) - arctan(1/{})"
    while True:
        yield template.format(exp1, exp2, next(a005248))
        exp1 -= 2
        exp2 -= 2

expr_gen = build_expr()

for _ in range(20):
    expr = next(expr_gen)
    print(expr)
    print(eval(expr))

(arctan( Ø ** 1) -  arctan( Ø ** -1)) - arctan(1/2)
0.0
(arctan( Ø ** -1) -  arctan( Ø ** -3)) - arctan(1/3)
0.0
(arctan( Ø ** -3) -  arctan( Ø ** -5)) - arctan(1/7)
0.0
(arctan( Ø ** -5) -  arctan( Ø ** -7)) - arctan(1/18)
-1.3877787807814457e-17
(arctan( Ø ** -7) -  arctan( Ø ** -9)) - arctan(1/47)
0.0
(arctan( Ø ** -9) -  arctan( Ø ** -11)) - arctan(1/123)
-3.469446951953614e-18
(arctan( Ø ** -11) -  arctan( Ø ** -13)) - arctan(1/322)
-8.673617379884035e-19
(arctan( Ø ** -13) -  arctan( Ø ** -15)) - arctan(1/843)
-6.505213034913027e-19
(arctan( Ø ** -15) -  arctan( Ø ** -17)) - arctan(1/2207)
-2.168404344971009e-19
(arctan( Ø ** -17) -  arctan( Ø ** -19)) - arctan(1/5778)
-1.0842021724855044e-19
(arctan( Ø ** -19) -  arctan( Ø ** -21)) - arctan(1/15127)
-2.710505431213761e-20
(arctan( Ø ** -21) -  arctan( Ø ** -23)) - arctan(1/39603)
-2.0328790734103208e-20
(arctan( Ø ** -23) -  arctan( Ø ** -25)) - arctan(1/103682)
-8.470329472543003e-21
(arctan( Ø ** -25) -  arctan( Ø ** -27)) - a

Looking good!